# EDA

In [ ]:
 • 문자열 formatting 2가지

In [ ]:
print(‘The story of {0},{1}, and {other}’.format(‘Bill’,’Manfred’,other=’George’))
print(f’I spent {h} hours coding {lang}’)

■ Pandas(with Numpy) 

In [ ]:
import pandas as pd

• 불러오기

In [ ]:
import os  # os에 캐글 한줄 zipfile한줄
import zipfile
os.system('kaggle competitions download -c titanic -p C:\\Users\\Dongjun\\.kaggle') #Including the part downloading the data
zipfile.ZipFile('C:\\Users\\Dongjun\\.kaggle\\titanic.zip').extractall('C:\\Users\\Dongjun\\.kaggle\\titanic') 
titanic_df=pd.read_csv(’C:\\Users\\john\\titanic_train.csv’) #csv는 ,로 구별한 파일양식 

• 만들기

In [ ]:
df=pd.DataFrame(ndarray,columns=['col1','col2']) 
s=pd.Series(ndarray) #가끔 index불일치로 오류뜰때가 있다
ndarray=df.values #ndarray가 필요한 작업들이 있다(특히 columns series를 for루프에 넣을 리스트로 만들때)

• Numpy(list약간확장)

In [ ]:
ndarray=np.array([1,2,3]) #comprehension없이 빈리스트 np.zeros(3,2)나 np.arange(3,7)처럼 쉽게 생성가능
array2=array1.reshape(2,3) #차원변경 #-1넣으면 호환되는 값을 대신 대입 #없으면 comprehension으로 하면 됨

In [ ]:
sort_indices=np.argsort(score_array) 
print(name_array[sort_indices]) #메타데이터가 없는 ndarray에서 활용도 높음

In [ ]:
np.dot(A,B) 
np.transpose(A)

## Basic statistics

In [ ]:
titanic_df.head(3) #직접 보기
titanic_df.info() #null개수파악, dtype보고 encoding계획 

In [ ]:
titanic_df.drop(['Age','Sex'],axis=1,inplace=True) #axis와 inplace필수, 잠깐만 뺀다는 점에서 inplace=False도 유용

## Basic feature engineering

In [ ]:
titanic_df['New_feature']=[1,2,3] #칼럼 추가
titanic_df['Family']=titianic_df['SibSp']*10+titanic_df['Parch']
titanic_df['Name_len']=titanic_df['Name'].apply(lambda x: 'adult' if x < 20 else 'not_adult') #len(titanic['Name])이 아니라 apply lambda

• 불린 인덱싱

In [ ]:
titanic_boolean=titanic_df[(titanic_df['Age']>60)&(titanic_df['Sex']=='female')] #새로운 dataframe만들때
df.loc[(df.Age>15)&(df.Age<=30), 'Age_band']=2 #새로운 feature만들때
df.iloc[-251:-1,2:4] #이름을 직접적기 불편할때  

# Preprocessing

가장 먼저 dtype먼저 분류<br>
mixed는 미리 분리해 각각의 feature로 만들기 

In [ ]:
categorical=[var for var in df.columns if df[var].dtype=='o'] #순서있으면 ordinal,없으면 nominal(제일먼저 제거대상)
discrete=[var for var in df.columns if df[var].dtype!='o' and var!='Survived' and df[var].nunique()<10] #categorical 
continuous=[var for var in df.columns if df[var].dtype!='o' and var!='Survived' and var not in discrete]

- Visualization
sns는 dataframe에 맞춘 형태(feature명입력)<br>
plt는 좌표입력

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig=plt.figure(figsize=(10,5)) #계속 똑같은 형태로 구현하기
fig.add_subplot(311)
sns.countplot(data=df,x='Pclass',hue='Sex',ax=ax[0])  # pclass기준으로 group, hue추가(sns장점)
sns.histplot(data=df,x='Age',ax=ax[0]) #displot안되니까 hist,kde로 나눌수 밖에
sns.kdeplot(datat=df,x='Age',ax=ax[0])
sns.barplot(data=df,x='Cabin',y='Survived',ax=ax[0])  # 오차범위 막대도 같이 나옴
sns.scatterplot(data=df,x='Age',y='Survived')
df.plot.pie()
df.plot.bar()  
df.plot.hist()  # continuous한 data를 bar처럼
df.plot.kde()
plt.title('Real')
plt.ylabel('Price')
plt.yticks(range(50,100,10),rotation=60,fontsize='medium')
plt.tight_layout() #간격조절은 항상 마지막
plt.show()

## Imputation
모든 var의 문제/모든 model에 필수<br>
- 5%보다 크면 not at random 가능성 커지니<br>
numerical은 median+indicator추가(skewed일때 median이 최선)<br>
categorical은 frequent+missing label<br>
해서 원인 보정<br>
- 작으면 median/frequent으로 끝냄<br>
- 수집과정 분석이용 arbitrary imputation도 괜찮<br>
cf)End of tail 방식은 보통 99,-99등으로 해두는 게 국룰<br>
+KNNImputer나 IterativeImputer도 똑같이 활용가능(Multivariate)

In [ ]:
df.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer #Sklearn패키지는 numpy반환해서 살짝 불편
imputer=SimpleImputer(strategy='median') #strategy=most_frequent,strategy='constant',fil_value=-99
df[columns]=imputer.fit_transform(df[columns]) 
from sklearn.impute import MissingIndicator
indicator=MissingIndicator().fit_transform(df['Cabin_cat'].values.reshape(-1,1)) #only ndarray
df['Cabin_ind']=pd.Series(indicator.reshape(-1)) #to 1dim

In [ ]:
from sklearn.impute import KNNImputer
knn_imp=KNNImputer(n_neighbors=2)
ndarray=df['Cabin'].values
imputed=knn_imp.fit_transform(ndarray)
imputed_df=pd.Series(imputed,index=df.index)

## Encoding
category만의 문제/모든 model에 필수<br>
category를 numerical로 바꿔주는 작업(기계적으로)<br>

ordinal이면 LabelEncoder<br>
nominal이면 
- Onehotencoding of top categories(정보손실있지만 feature space 너무 커지는 거 방지)<br>
- Frequency encoding(빈도까지 고려가능)
- monotonic함 반영한 encoding(linear model들이 좋아함)<br>
weight of evidence 활용한 방식도 있음(ln(good%/bad%))


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder().fit_transform(df[ordinal])

In [ ]:
top10=[x for x in df[var].value_counts.sort_values(ascending=False).head(10).index]
df = pd.get_dummies(df,columns=top10)  # inplace안됨

In [ ]:
dict_title_age=temp.groupby('Title')[['Age']].median().to_dict()
dic=df['Neighborhood'].value_counts().to_dict() # series의 인덱스와 값을 딕셔너리로 반환
df['Neighborhood']=df['Neighborhood'].map(dic)

In [ ]:
label=df.groupby(['Neighborhood'])['SalePrice'].mean().sort_values(ascending=False).index #groupby로 만들면 index애용하자
ordinal_dict={k:i for i,k in enumerate(label)} #dict도 이렇게 가능
df['Neighborhood_mono']=df['Neighborhood'].map(ordinal_dict)

## Discretization/Scaling

• Discretization<br>
category만의 문제/Tree기반model들이 좋아함<br>
Cardinality&Rare labels -> train,test중 하나에만 들어가서 overfitting<br>
- Encoding까지 같이하거나<br>
- Tree로 monotonic하게 만들거나(Linear model들만 좋아함)<br>
- 직접 설정해주거나<br>

In [ ]:
df[categorical+mixed].nunique() #cardinality,rare labels조사

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer #Encoding까지 같이함
discretizer=KBinsDiscretizer(n_bins=6,encode='ordinal',strategy='quantile') #encode는 ordinal,onehot #strategy=uniform(width),quantile(sample,유일하게 spread 향상),kmeans 

In [ ]:
clf=DecisionTreeClassifier(max_depth=5) #feature하나와 target관계분석
clf.fit(df['Neighborhood']).to_frame(),df['SalePrice'])
df['Age_disc']=clf.predict_proba(df['Age'].to_frame())[:,1] #target확률
disc=pd.concat([df.groupby('Age_disc')['Age'].min(),df.groupby('Age_disc')['Age'].max()],axis=1)

In [ ]:
df['Age_bins']=pd.cut(df['Age'],bins=[20,40]) #직접 설정해주기

• Scaling<br>
numerical만의 문제/Tree기반model빼고 좋아함<br>
target은 필수, 분포가 편향적인 것을 정규분포가깝게(feature끼리 scale 다르면 coef바뀜(feature importance 왜곡,속도문제)<br>
Outlier 적으면 직접 평균값으로 바꿔주기도 함 

In [ ]:
titanic_df.describe() #눈으로 outlier확인가능
sns.boxplot(data=df,x=[discrete+continuous],y='Chance of Admission')

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaled_df=MinMaxScaler().fit_transform(df)
df['count']=np.log1p(df['count']) #마지막 pred값을 expm1로 돌려놓기

In [ ]:
quantile_75=df['windspeed'].quantile(0.75)
iqr=quantile_75-quantile_25
df['windspeed'].loc(df['windspeed']>quantile_75+iqr*1.5)=df['windspeed'].mean()

## Feature selection


• SelectKBest<br>
target과 각 feature의 통계학적 연관성<br>
- chi2:categorical feature(mutual_info_classif도 가능)<br>
수학적으로 category값별 target값의 실제값,예측값 비교<br>
- f_classif:numerical feature<br>
수학적으로 target별 numerical값의 평균이 다름(ANOVA방식)

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif,f_classif
X=SelectKBest(chi2,k=2).fit_transform(X,y) #SelectPercentile(chi2,percentile=10)도 괜찮

In [ ]:
df.corr.where(np.triu(np.ones(df.corr)).astype(np.bool_),inplace=True) #upper corr로 만들기(True,1이면 남김) #triangular upper #bool means trueorfalse
to_drop=[column for column in upper.columns if any(upper[column]>0.9)] #[return값 for문 똑같음 if조건문] #corr matrix를 column단위로 분석
df.drop(to_drop,inplace=True)

• SequentialFeatureSelector<br>
-Recursive<br>
-Forward/Backward<br>

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs=SFS(RandomForestRegressor(),forward=False,scoring='neg_mean_squared_error',k_features=10,cv=3)

• coef_/feature_importances_<br>

In [ ]:
sel=SelectFromModel(Lasso(alpha=100))
sel.fit(X,y)
X=X[sel.get_support()] #get_support는 객체반환(직접해봐야알듯)

In [ ]:
importances=pd.Series(rf_clf.feature_importances_,index=X.columns) #자체로는 ndarray
importances.sort_values()
sns.barplot(x=importances,y=importances.index)

• Dimension reduction(Too many features)<br>
- PCA<br>
공분산행렬(symmetric)을 EVD, 고유값 첫번째, 두번째큰 고유벡터로 분해<br>
- Truncated SVD<br> 
PCA와 다르게 rectangular로 확장<br>
∑최소로 diagonal하게 만들고 U,Vt도 맞춰주기<br>
- LDA<br>
클래스간 분산최대, 클래스내 분산최소


In [ ]:
from sklearn.decomposition import PCA, Truncated SVD 
from sklearn.dicriminant_analysis import LinearDiscriminantAnalysis
pca=PCA(n_components=6)
santander_pca=pca.fit_transform(santander_scaled) #보통 scale하고 적용 #ndarray반환
pca_columns=['pca1','pca2','pca3','pca4','pca5','pca6']
santander_df=pd.DataFrame(santander_pca,columns=pca_columns)

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV, KFold, StratififedKFold #Stratified는 label분포가 치중되었을때(주로 분류)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(titanic_feature,titanic_label,test_size=0.2,random_state=0) #X가 먼저 #재현을 위해 randomstate필수

In [ ]:
kfold=KFold(n_splits=5,random_state=0) #5개의 kfold객체 생성 
for train_index,test_index in kfold.split(titanic_feature,titanic_label): #각 인덱스를 ndarray로 반환 
  X_train,X_test=titanic_feature[train_index], titanic_feature[test_index]
  y_train,y_test=titanic_label[train_index], titanic_label[test_index]
  lgbm_clf.fit(X_train,y_train)
  cv_accuracy.append(accuracy_score(y_test,lgbm.predict(X_test)))
  print('cv정확도:',cv_accuracy)
print('\n평균cv정확도:',np.mean(cv_accuracy))

■ Tree기반

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

• RandomForest(Bagging)<br>
병렬적으로 결정트리에 bootstrapping된 데이터 학습<br>
DecisionTree기반 알고리즘은 회귀도 가능(exRandomForestRegressor)

In [ ]:
rf_clf=RandomForestClassifier(random_state=0) #딕셔너리 방식으로 입력
params={'n_estimators':[100] #100개의 tree에서 나온 확률값 평균
        'max_depth':[6,8,10] #이하 3개는 결정트리것 그대로
        'min_samples_leaf':[8,12] #samples수 너무 적을 때까지 않가도록(클수록 과적합방지)
        'min_samples_split':[8,16] #split기준은 지니계수(한쪽이 우세한 정도라 직관적)
        }
rf_gridcv = GridSearchCV(rf_clf,param_grid=params,cv=5)
rf_gridcv.fit(X_train,y_train)

• XGBoost(Boosting)<br> 
GBM대비 병렬학습가능해 빠르고 과적합규제/가지치기로 split재검토,내부적 교차검증,결손값처리,조기중단

In [ ]:
xgb_clf=XGBClassifier(random_state=0)
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[3,5,7],
        'colsample_bytree':[0.7,0.8],
        'subsample':[0.9]}

• LightGBM(Boosting)<br> 
XGBoost와 달리 leafwise 분할(시간,공간복잡도 개선, 성능비슷)

In [ ]:
lgbm_clf=LGBMClassifier(random_state=0)
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[3,5,7], #leafwise니까 살짝 더 크게
        'colsample_bytree':[0.7,0.8], 
        'subsample':[0.9]
        }
grid_cv.fit=GridSearchCV(lgbm_clf,param_grid=params,cv=5)
grid_cv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric='auc',eval_set=[(X_train,y_train),(X_test,y_test)] #향상되지 않고 반복할 수 있는 최대횟수 #eval_set은 보통 학습데이터셋말고 다른것 

■ 분류 

• LogisticRegression<br>
sigmoid함수반환값을 확률로 간주<br>
penalty로 규제유형(디폴트는 l2) / C로 alpha역수

• SVC<br>
feature space에서 가장 넓은 도로를 놓을 수 있는 선(large marginal classifictation)<br>
도로안에 가장 많은 sample들어가게 하는 선으로 회귀도 가능(파라미터는 epsilon=1.5)<br>
kernel trick방식은 속도가 좀 느림(유사도나 다항처리나)
비교적 작은 데이터셋에 적합

In [ ]:
from sklearn.svm import LinearSVC,SVC
svm_clf=LinearSVC(C=1,loss='hinge') #C는 violation, 낮을수록 설명력 향상

In [ ]:
svm_clf=SVC(kernel='rbf',C=0.001,gamma=5) #gamma는 높을수록 설명력 향상(각 landmark의 영향력) #landmark와의 유사도(RadialBasisFunction)

• Metric<br>
confusion matrix에서 FP와 FN줄이기

In [ ]:
from sklearn.metrics import recall_score,precision_score,f1_score,roc_auc_score
def get_clf_eval(real,pred):
  recall=recall_score(real,pred) #FP줄이고 싶음(암진단모델)
  precision=precision_score(real,pred) #FN줄이고 싶음(스팸차단모델)
  f1=f1_score(real,pred) #치우치지 않은 정도
  auc=roc_auc_score(real,pred) #TPR-FPR그래프에서 TPR이 얼마나 큰지(1에 가까울수록 우수)
  print('recall score:{0:.4f} / precision score:{1:.4f} / f1 score:{2:.4f} / f1 score:{3:.4f}'.format(recall,precision,f1,auc)) 

■ 회귀

• PolynomialFeatures<br>
주어진 변수로 파생변수추가해 설명력향상 but overfit 위험, 변수 너무 많아짐(커봤자 degree3)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
feature_df=PolynomialFeatures(degree=3).fit_transform(feature_df) 

• Regulation<br>

In [ ]:
from sklearn.linear_model import Ridge,Lasso,ElasticNet
Ridge(alpha=0.1).fit(X_train,y_train) #커질수록 설명력 감소 #L2:계수크기규제(RSS+alpha*W2최소화)
Lasso(alpha=0.1).fit(X_train,y_train) #L1:계수종류규제->feature selection기여(RSS+alpha*W최소화)
ElasticNet(alpha=0.1,l1_ratio=0.7).fit(X_train,y_train) #alpha는 합 

• Metric<br>
MSE, R2

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score #MSE와 R2
def get_clf_eval(real,pred):
  mse=mean_squared_error(real,pred)
  r2=r2_score(real,pred)
  print('MSE:{0:.4f} / R2:{1:.4f}'.format(mse,r2)) 

■ 군집화

• Aggregation<br>
groupby객체는 새index 기준 group한 것(1~2개)<br>
특정 feature를 aggregate해서 보여주거나<br>
여러 feature해서 새 dataframe만들거나<br>

In [ ]:
cust_df.groupby(['target','cluster']).size()  # target과 cluster 관계분석
iris_df.groupby(['CustomerID']).agg({'InvoiceDate':'max','InvoiceNo':'count','sale_amount':'sum'})

• KMeans<br>
centroid별 평균중심으로 업데이트

In [ ]:
from sklearn.cluster import KMeans
kmeans_cls=KMeans(n_clusters=3,max_iter=300)
label=kmeans_cls.fit_predict(test_df)
iris_df['cluster']=pd.Series(label,index=iris_df.index) #index설정안해서 한시간 버림
iris_df.groupby(['target','cluster'])['slength'].count #꼭 두개씩 group해야 의미있음,count써야함(groupby객체반환) 

• MeansShift<br>
centroid별 밀도중심으로 업데이트(KernelDensityFunction)

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth
meanshift_cls=MeanShift(bandwidth=estimate_bandwidth) #최적의 bandwidth값 return(작아야 설명력 향상)

• DBSCAN<br>

In [ ]:
from sklearn.cluster import DBSCAN #min_samples 넘는 corepoint끼리 연결/적으면 borderpoint거나 noise
dbscan=DBSCAN(eps=0.6,min_samples=8,metric='euclidean') #eps클수록 border많아짐/min_samples클수록 core적어짐

• 시각화<br>
무조건 2개로 차원축소

In [ ]:
pca_transformed=pca.fit_transform(iris_df)
iris_df['x']=pca_transformed[:,0]
iris_df['y']=pca_transformed[:,1]
ind0=iris_df[iris_df['cluster']==0].index
ind1=iris_df[iris_df['cluster']==1].index
plt.scatter(x=iris_df.loc[ind0,'x'],y=iris_df.loc[ind0,'y'])
plt.scatter(x=iris_df.loc[ind1,'x'],y=iris_df.loc[ind1,'y'])

• Metric<br>
silhouette coef<br>
각 데이터마다 b-a/max(b,a) / a는 클러스터내 평균거리, b는 최근접 클러스터내 평균거리<br>
군집별 silhouette_sample이 관심


In [ ]:
from sklearn.metrics import silhouette_samples
scores=silhouette_samples(feature_df,iris_df['cluster']) #특이하게 feature와 label입력 #모든 데이터의 score저장(1일수록 우수)
iris_df['score']=pd.Series(scores,index=iris_df.index)

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor
vo_reg = VotingRegressor(estimators=[('lr',lr_reg),('ridge',lr_ridge),('lasso',lr_lasso)])  # 의외로 이름이 필수 인수
vo_reg.fit(X_train,y_train)
pred4 = vo_reg.predict(X_test)